In [3]:
# normal stuff
import pandas as pd
import numpy as np
import cPickle as pickle
pd.set_option('max_colwidth', -1)

import string
import re
import codecs

# for scraping
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from bs4 import BeautifulSoup
import re
import requests
import shutil



C:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Load MyDramaList URL list

Load the list of MyDramaList URLs for each show.

In [4]:
# read in the url list
show_index = pd.read_csv('./dramalist-urls.txt', delimiter='\t')
show_index.dropna(inplace=True)
show_index.reset_index(drop=True, inplace=True)
print show_index.shape
show_index

(386, 2)


,showname,url
0,ma boy,https://mydramalist.com/5104-ma-boy
1,falling for innocence,https://mydramalist.com/12360-falling-for-innocence
2,mad dog,https://mydramalist.com/24187-mad-dog
3,to the beautiful you,https://mydramalist.com/2278-to-the-beautiful-you
4,vampire prosecutor 2,https://mydramalist.com/4920-vampire-prosecutor-2
5,joseon x-files,https://mydramalist.com/730-secret-investigation-record
6,beautiful gong shim,https://mydramalist.com/17219-beautiful-gong-shim
7,fashion king,https://mydramalist.com/3228-fashion-king
8,mysterious il-seung,https://mydramalist.com/24774-doubtful-victory
9,the moon that embraces the sun,https://mydramalist.com/3079-the-moon-that-embraces-the-sun


In [5]:
# there is a problem with encoding (see index 16). fix the encoding
import unicodedata
def fix_encoding(text):
    return text.decode('cp1252')

show_index.showname = show_index.showname.map(fix_encoding)

In [88]:
# check
show_index.showname[16]

u'i\u2019m not a robot'

In [99]:
show_index

,showname,url
0,ma boy,https://mydramalist.com/5104-ma-boy
1,falling for innocence,https://mydramalist.com/12360-falling-for-innocence
2,mad dog,https://mydramalist.com/24187-mad-dog
3,to the beautiful you,https://mydramalist.com/2278-to-the-beautiful-you
4,vampire prosecutor 2,https://mydramalist.com/4920-vampire-prosecutor-2
5,joseon x-files,https://mydramalist.com/730-secret-investigation-record
6,beautiful gong shim,https://mydramalist.com/17219-beautiful-gong-shim
7,fashion king,https://mydramalist.com/3228-fashion-king
8,mysterious il-seung,https://mydramalist.com/24774-doubtful-victory
9,the moon that embraces the sun,https://mydramalist.com/3079-the-moon-that-embraces-the-sun


# scraping dramalist

Using Selenium and BeautifulSoup, scrape mydramalist.com for metadata of each show. 

#### Fields scraped:
1. Num of episodes
2. Duration 
3. Score
4. Genres
5. Tags
6. Synopsis
7. Poster
8. top 6 actors
9. screenwriters
10. directors
11. Aired date

In [124]:
# define function to save image files
def save_image_to_file(image, dirname, suffix):
    with open('{dirname}/img_{suffix}.jpg'.format(dirname=dirname, suffix=suffix), 'wb') as out_file:
        shutil.copyfileobj(image.raw, out_file)

In [127]:
# initialise the webdriver
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')
wait = WebDriverWait(driver, 15)
driver.set_page_load_timeout(60)

# initialise the dictionary to collect metadata from all shows
all_shows = {}

# loop through all the show indices
for i in show_index.index: #[0, 1, 2]: 
    
    # open the show
    driver.get(show_index.url[i])
    sleep(1)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    
    # create new entry
    entry = {}
    
    # get poster
    url = soup.find('img', attrs={'itemprop':'image'}).get('src').encode('utf-8')
    response = requests.get(url, stream=True)
    save_image_to_file(response, 'images', i)

    # get list of genres
    entry['genres'] = [genre.strip() for genre in soup.find('li', attrs={'class':'show-genres'}).text.split(':').pop().split(',')]

    # get synopsis
    entry['synopsis'] = soup.find('div', attrs={'class': 'show-synopsis'}).text.strip()

    # get score
    entry['score'] = float(soup.find('div', attrs={'class': 'col-film-rating'}).text.strip())

    # loop through all the show details
    for li in soup.find('div', attrs={'class': 'show-detailsxss'}).find_all('li'):
        # get no of episodes
        if u'Episodes:' in li.text:
            entry['ep_no'] = int(li.text.split(':')[-1])

        # get duration of episodes
        if u'Duration:' in li.text:
            durs = li.text.split(':').pop().split('.')
            entry['ep_dur']=0
            for part in durs:
                if u'hr' in part:
                    entry['ep_dur'] += [int(sub)*60 for sub in part.split() if sub.isdigit()][0]
                if u'min' in part:
                    entry['ep_dur'] += [int(sub) for sub in part.split() if sub.isdigit()][0] 

        # get tags
        if u'Tags:' in li.text:
            entry['tags'] = [tag.strip() for tag in li.text.split(':').pop().split(',')]
            
        # get aired date
        if u'Aired:' in li.text:
            entry['aired_date'] = li.text.split(':')[-1].strip()

        # get directors
        if u'Directors:' in li.text:
            entry['directors'] = li.text.split(':')[-1].strip().split(', ')

        # get screenwriters
        if u'Screenwriters:' in li.text:
            entry['screenwriters'] = li.text.split(':')[-1].strip().split(', ')

    # get top 6 actors
    actors = []
    for actor in soup.find_all('div', attrs={'itemprop':'actor'}):
        actors.append(actor.b.text.replace('  ', ' ').strip())
    entry['actors'] = actors
    
    # show name
    entry['showname'] = show_index.showname[i]

    # append to main dictionary
    all_shows[i] = entry
    
    # show status
    print i,

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [129]:
# convert to dataframe format for easier viewing
df = pd.DataFrame(all_shows).T

In [134]:
# save to pickle file!
pickle.dump(df, open('dramalist_info.pkl', 'wb'))

In [135]:
# check
pickle.load(open('dramalist_info.pkl', 'rb'))

,actors,aired_date,directors,ep_dur,ep_no,genres,score,screenwriters,showname,synopsis,tags
0,"[Kim So Hyun, Sun Woong, Min Hoo, Hong Yoon Hwa, Kang Hyun Joong, Nam Tae Boo]","Aug 16, 2012 - Aug 30, 2012",NaN,60,3,"[Friendship, Comedy, Romance, School, Youth]",7.5,NaN,ma boy,"Geu Rim is a student at an arts high school who dreams of becoming a pop singer. At school She gets a new roommate, Irene who’s actually a boy. Irena’s persona is a pretty Female CF star and his actual name is Hyun Woo.","[Cross-dressing, Cohabitation, High School, Mini Drama]"
1,"[Kim So Yeon, Jung Kyung Ho, Yoon Hyun Min, Gong Hyun Joo, Jo Eun Ji, Uhm Hyo Sub]","Apr 3, 2015 - May 23, 2015",NaN,60,16,"[Comedy, Romance, Drama, Melodrama]",8.4,NaN,falling for innocence,"Min-Ho's father ran a large company, but his death led to his uncle taking over the company and to Min-Ho's growing into a cutthroat investor. But then his personality changes when the merciless corporate head hunter undergoes a heart transplant surgery. Thanks to his new heart, Min-Ho begins to have warm feelings and he falls in love with a woman named Soon-Jung.","[Love Triangle, Rich Male Lead, Illness, Boy Chases Girl First]"
2,"[Yoo Ji Tae, Woo Do Hwan, Ryu Hwa Young, Jo Jae Yun, Kim Hye Sung, Hong Soo Hyun]","Oct 11, 2017 - Nov 30, 2017",NaN,60,16,"[Thriller, Romance, Crime, Investigation]",8.5,NaN,mad dog,"A realistic drama about the team “Mad Dog”, which catches scam artists that commit insurance fraud.\nThe drama portrays the reality of Korea through diverse insurance crimes that are prevalent in the country. There are the 'copy cats' that imitate and improve the existing crimes, and the 'mad dogs' who catch these bad cats.",NaN
3,"[Sulli, Choi Min Ho, Lee Hyun Woo, Kim Ji Won, Seo Joon Young, Kang Ha Neul]","Aug 15, 2012 - Oct 4, 2012",[Jun Ki Sang],64,16,"[Comedy, Romance, School, Youth, Drama, Sports]",7.6,NaN,to the beautiful you,"Goo Jae Hee, is a Korean girl who lives in the United States. One day, she sees a track and field competition on TV, and becomes attracted to one of the high jump competitors, Kang Tae Joon. She begins to idolize the young athlete and in order to help him, she disguises herself as a boy and transfers to his all-male high school after he suffered an accident that could potentially end his career.","[Idol, Cross-dressing, Adapted From A Manga, Celebrity, High Jump]"
4,"[Yeon Jung Hoon, Lee Young Ah, Lee Won Jong, Kim Joo Young, Lee Kyung Young, Gong Jung Hwan]","Sep 9, 2012 - Nov 18, 2012",NaN,65,11,"[Action, Mystery, Vampire, Drama, Investigation]",8.5,[Han Jung Hoon],vampire prosecutor 2,"In the second season of “Vampire Prosecutor,” Min Tae Yeon is back as the prosecutor who heads up a new division that investigates unsolved murder cases. No one, except for Detective Hwang Soon Bum, knows that Tae Yeon was bitten seven years ago and acquired vampire powers. Together, they solve murder cases, and each time Tae Yeon uses his “powers” to find leads on cases that seem unsolvable. But the one thinghe can’t solve is: who was the vampire/serial killer that turned him seven years ago? And what happens when Jung In becomes suspicious of Tae Yeon’s “questionable” tactics in solving cases.",NaN
5,"[Kim Gab Soo, Im Jung Eun, Jo Hee Bong, Kim Ji Hoon, Jun So Min, Jo Jung Eun]","Aug 20, 2010 - Oct 29, 2010",[Kim Heung Dong],45,12,"[Historical, Mystery, Sci-Fi]",7.8,NaN,joseon x-files,"Secret Investigation Record is based on cases detailed in the Annals of Joseon History with a bizarre twist; these incidents incorporate mysterious, supernatural elements that cannot be explained through science. Kim Hyung Do is an inspector of the state whose single-minded focus on his investigations into administrative misdeeds bring him into direct contact with these strange mysteries. Heo Yoon Yi is a government employee who aids him and attempts to understand thesescientific impossibilities with an open mind, in contrast to his cynicism.",[Joseon Dynasty]
6,"[Nam Goong Min, Bang